In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import set_config
set_config(transform_output='pandas')
from sklearn.impute import SimpleImputer

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate,StratifiedKFold, cross_val_score, train_test_split
import os
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder,MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor

In [2]:
os.chdir('C:/Users/Katharina/Desktop/Weiterbildung/Bootcamp/Bootcamp/Final_project/data')
data = pd.read_csv('ML_Data_Teams.csv', sep=';')	

In [3]:
data1 = data[['@PositionInEntry',
       '@Rank', '@EarnedPointsTeam', '@EarningsTotalTeam', 'Gender_x', 'Type',
       'SpikeFault', 'SpikePoint', 'ServeFault', 'ServePoint',
       'ServeTotal', 'BlockPoint', 'BlockTotal', 'DigTotal', 'ReceptionFault',
       'SpikeTotal', '@LocalDate', '@LocalTime', 'FederationCode_y',
       'FirstName', 'LastName', 'FirstName2', 'LastName2',   '@PointsTeamASet1',
       '@PointsTeamBSet1', '@PointsTeamASet2', '@PointsTeamBSet2',
       '@PointsTeamASet3', '@PointsTeamBSet3', '@DurationSet1',
       '@DurationSet2', '@DurationSet3', 'temperature_2m', 'precipitation',
       'wind_speed_10m', 'rain', 'wind_gusts_10m', 'TeamFault_team',
       'match_win',  'Team1', 'Team2']]
#'TournamentNo', 

In [4]:
y = data1.pop('SpikePoint')
X = data1.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

In [5]:
special_feature = '@DurationSet3'

# Kopiere die Trainingsdaten, um das Original nicht zu überschreiben
X_train = X_train.copy()

# Erzeuge den Indikator: 1, wenn in @DurationSet3 ein Wert vorhanden ist, sonst 0.
X_train[special_feature + '_indicator'] = X_train[special_feature].notnull().astype(int)

# Für die Testdaten ebenfalls:
X_test = X_test.copy()
X_test[special_feature + '_indicator'] = X_test[special_feature].notnull().astype(int)

In [6]:
nominal_features = X_train.select_dtypes(include=['object']).columns.tolist()
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Erstelle eine Liste der allgemeinen numerischen Features, exklusive des speziellen Features und dessen Indikator
general_numeric_features = [col for col in numeric_features if col not in [special_feature, special_feature + '_indicator']]

# Erstelle separate Pipelines:
# 1. Für allgemeine numerische Features mit KNNImputer
numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler())
])

# 2. Für das spezielle Feature, das immer mit 0 imputiert werden soll (das signalisiert, dass kein dritter Satz stattgefunden hat)
special_numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler())
])

# 3. Für die Indikatorspalte (@DurationSet3_indicator)
indicator_transformer = Pipeline(steps=[
    # Hier ist meist keine Imputation nötig; Skalierung ist optional, da 0 und 1 oft schon aussagekräftig sind.
    ('scaler', StandardScaler())
])

# 4. Für nominale Features: Imputation mit dem häufigsten Wert + OneHotEncoding
nominal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Baue den ColumnTransformer unter Berücksichtigung aller Feature-Gruppen:
transformers = [
    ('num', numeric_transformer, general_numeric_features),
    ('spec_num', special_numeric_transformer, [special_feature]),
    ('indicator', indicator_transformer, [special_feature + '_indicator']),
    ('nom', nominal_transformer, nominal_features)
]

preprocessor = ColumnTransformer(transformers=transformers)

In [7]:
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error

model_scores = []
def score_model(model_name, y_true, y_pred):
  scores = {
      'Model': model_name,
      'MAE ($)': round(mean_absolute_error(y_true, y_pred), 2),
      'RMSE ($)': round(root_mean_squared_error(y_true, y_pred), 2),
      'MAPE (%)': round(100 * mean_absolute_percentage_error(y_true, y_pred), 2),
      'R-Squared': round(r2_score(y_true, y_pred), 3)
  }
  return scores

In [8]:
from sklearn.ensemble import GradientBoostingRegressor
gbr_pipeline = make_pipeline(preprocessor,
                             RobustScaler(),
                             GradientBoostingRegressor(random_state=42, n_estimators=200, learning_rate=0.1, 
                                                  max_depth=5, subsample=0.8))

GB = gbr_pipeline.fit(X_train, y_train)

gbr_predictions = gbr_pipeline.predict(X_test)

model_scores.append(score_model('GradientBoostigRegressor', y_test, gbr_predictions))
pd.DataFrame(model_scores)

,Model,MAE ($),RMSE ($),MAPE (%),R-Squared
0,GradientBoostigRegressor,2.07,2.64,2.769038e+15,0.861


In [9]:
train_r2 = gbr_pipeline.score(X_train, y_train)
test_r2  = gbr_pipeline.score(X_test, y_test)
print("Train R²:", train_r2)
print("Test R²:", test_r2)

Train R²: 0.922848665048692
Test R²: 0.8612781680769611


In [10]:

# 1. Extrahiere das trainierte Model (der letzte Schritt in der Pipeline)
model = gbr_pipeline.named_steps['gradientboostingregressor']

# 2. Extrahiere den Preprocessor-Schritt, der die Features formatiert hat.
# Je nachdem, wie deine Pipeline aufgebaut ist, kann der Schritt-Name variieren.
# Beim make_pipeline werden die Namen automatisch generiert:
preprocessor = gbr_pipeline.named_steps[list(gbr_pipeline.named_steps.keys())[1]]
# Alternativ, wenn du die Pipeline explizit mit Namen erstellt hast, z.B. im Pipeline-Konstruktur,
# dann wäre es etwas wie:
preprocessor = gbr_pipeline.named_steps['columntransformer']

# 3. Hole die Feature-Namen aus dem Preprocessor:
# Diese Methode funktioniert, wenn der Preprocessor und seine Zwischenschritte get_feature_names_out unterstützen.
feature_names = preprocessor.get_feature_names_out()

# 4. Extrahiere die Feature Importances aus dem Model
importances = model.feature_importances_

# 5. Erstelle einen DataFrame zum besseren Überblick
feature_importances_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print(feature_importances_df.head(10))


                    feature  importance
9           num__ServeTotal    0.684573
14          num__SpikeTotal    0.120599
28      num__TeamFault_team    0.052471
10          num__BlockPoint    0.013295
12            num__DigTotal    0.012786
6           num__SpikeFault    0.012014
8           num__ServePoint    0.011074
29           num__match_win    0.009241
4             num__Gender_x    0.005062
3   num__@EarningsTotalTeam    0.003876


In [11]:
print(gbr_pipeline.named_steps['gradientboostingregressor'].get_params())

{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'squared_error', 'max_depth': 5, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_iter_no_change': None, 'random_state': 42, 'subsample': 0.8, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}


In [12]:
from sklearn.ensemble import RandomForestRegressor
rf_pipeline = make_pipeline(preprocessor,
                             RobustScaler(),
                             RandomForestRegressor(random_state=42, n_estimators=200, max_depth=15, 
                                         min_samples_split=5, min_samples_leaf=2))

RF = rf_pipeline.fit(X_train, y_train)

rf_predictions = rf_pipeline.predict(X_test)

model_scores.append(score_model('RandomForest', y_test, rf_predictions))
pd.DataFrame(model_scores)

,Model,MAE ($),RMSE ($),MAPE (%),R-Squared
0,GradientBoostigRegressor,2.07,2.64,2.769038e+15,0.861
1,RandomForest,2.26,2.92,2.562086e+15,0.831


In [13]:
print(rf_pipeline.named_steps['randomforestregressor'].get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': 15, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 200, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


In [14]:
train_r2 = rf_pipeline.score(X_train, y_train)
test_r2  = rf_pipeline.score(X_test, y_test)
print("Train R²:", train_r2)
print("Test R²:", test_r2)

Train R²: 0.9531795459779542
Test R²: 0.8307458799956113


In [15]:
# 1. Extrahiere das trainierte Model (der letzte Schritt in der Pipeline)
model = rf_pipeline.named_steps['randomforestregressor']

# 2. Extrahiere den Preprocessor-Schritt, der die Features formatiert hat.
# Je nachdem, wie deine Pipeline aufgebaut ist, kann der Schritt-Name variieren.
# Beim make_pipeline werden die Namen automatisch generiert:
preprocessor = rf_pipeline.named_steps[list(rf_pipeline.named_steps.keys())[1]]
# Alternativ, wenn du die Pipeline explizit mit Namen erstellt hast, z.B. im Pipeline-Konstruktur,
# dann wäre es etwas wie:

preprocessor = rf_pipeline.named_steps['columntransformer']

# 3. Hole die Feature-Namen aus dem Preprocessor:
# Diese Methode funktioniert, wenn der Preprocessor und seine Zwischenschritte get_feature_names_out unterstützen.
feature_names = preprocessor.get_feature_names_out()

# 4. Extrahiere die Feature Importances aus dem Model
importances = model.feature_importances_

# 5. Erstelle einen DataFrame zum besseren Überblick
feature_importances_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print(feature_importances_df.head(10))


                    feature  importance
9           num__ServeTotal    0.689218
14          num__SpikeTotal    0.106261
28      num__TeamFault_team    0.044339
6           num__SpikeFault    0.013349
8           num__ServePoint    0.009760
10          num__BlockPoint    0.009391
12            num__DigTotal    0.009348
23      num__temperature_2m    0.006637
3   num__@EarningsTotalTeam    0.006275
25      num__wind_speed_10m    0.006219
